In [295]:
import pandas as pd
import numpy as np
import pycountry


# Country table

In [296]:
# Get a list of all countries
countries_short = [country.alpha_2 for country in pycountry.countries]
countries_name = [country.name for country in pycountry.countries]

countries_df = pd.DataFrame({'handle': countries_short, 'name': countries_name}).reset_index()
countries_df = countries_df.rename(columns={"index":"country_id"})

# Movies Dataset cleaning

In [336]:
movies = pd.read_csv(r"C:\Users\Martijn\Downloads\netflix_titles_anandshaw.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [337]:
# How many rows with including the words "alien" and "ufo" in the title or description? 
filtered_df = movies[movies['title'].str.contains('alien|ufo|extraterrestrial|spaceship|spacecraft|cosmic|intergalactic|martian|extraterrestrials|galactic|asteroid|space|starship', case=False, na=False) | 
                     movies['description'].str.contains('alien|ufo|extraterrestrial|spaceship|spacecraft|cosmic|intergalactic|martian|extraterrestrials|galactic|asteroid|space|starship', case=False, na=False)]
print(len(filtered_df))

# create new column that indicates if movie has ufo theme
movies['ufo_theme'] = np.where(
    movies['description'].str.contains(
        'alien|ufo|extraterrestrial|spaceship|spacecraft|cosmic|intergalactic|martian|extraterrestrials|galactic|asteroid|space|starship', 
        case=False, 
        na=False
    ), 
    'yes', 
    'no'
)

movies.columns

174


Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'ufo_theme'],
      dtype='object')

In [ ]:
movies['date_added'] = pd.to_datetime(movies['date_added'], errors='coerce')
# Drop rows where 'date_added' is NaT due to coercion errors
movies = movies.dropna(subset=['date_added'])
movies = movies[['date_added', 'ufo_theme', 'release_year', 'type', 'title']]
movies['date_added_formatted'] = movies['date_added'].dt.strftime('%m/%Y')

# UFO reports cleaning

In [300]:
ufo_report  = pd.read_csv(r"C:\Users\Martijn\Downloads\nuforc_reports (1).csv")

In [302]:
ufo_report.info()

# rename columns to lower case
ufo_report.columns = ufo_report.columns.str.lower()

# remove columns 'Date', 'Posted', 'Shape', 'Duration', 'Image', 'Link', 'Summary', 'Text' (to avoid confusion about correct date column and other columns)
ufo_report = ufo_report.drop(columns=['date','posted', 'shape', 'duration', 'image', 'link', 'summary', 'text'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143940 entries, 0 to 143939
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Date_Table  143940 non-null  object
 1   Date        143940 non-null  object
 2   Posted      143940 non-null  object
 3   City        143933 non-null  object
 4   State       143895 non-null  object
 5   Country     143938 non-null  object
 6   Shape       143940 non-null  object
 7   Duration    143915 non-null  object
 8   Image       143940 non-null  object
 9   Link        143940 non-null  object
 10  Summary     143940 non-null  object
 11  Text        143906 non-null  object
dtypes: object(12)
memory usage: 13.2+ MB


In [303]:
# rename column date_table
ufo_report = ufo_report.rename(columns={'date_table': 'date'})
print(ufo_report)


                             date                city    state  country
0                         04/2023            Honolulu       HI      USA
1                         04/2023         Bakersfield  Unknown      USA
2                         04/2023         Castle Dale  Unknown  Unknown
3                         04/2023           Baltimore       MD      USA
4                         03/2023             Madison       WI      USA
...                           ...                 ...      ...      ...
143935                    07/1947             Roswell       NM      USA
143936                    06/1947      Corpus Christi       TX      USA
143937                    06/1952              Auburn       WA      USA
143938  UNSPECIFIED / APPROXIMATE           Troutdale       OR      USA
143939                    06/1950  Budapest (Hungary)  Unknown  Hungary

[143940 rows x 4 columns]


In [304]:
correct_format = r'^\d{1,2}/\d{4}$'

# count rows that do not have date format of MM/YYYY in 'Date_Table'
invalid_rows_count = ufo_report[~ufo_report['date'].str.match(correct_format, na=False)].shape[0]

print(f"invalid rows: {invalid_rows_count}")

# remove rows with invalid date format 
ufo_report = ufo_report[ufo_report['date'].str.match(correct_format, na=False)]
print(ufo_report)

invalid rows: 376
           date                city    state  country
0       04/2023            Honolulu       HI      USA
1       04/2023         Bakersfield  Unknown      USA
2       04/2023         Castle Dale  Unknown  Unknown
3       04/2023           Baltimore       MD      USA
4       03/2023             Madison       WI      USA
...         ...                 ...      ...      ...
143934  07/1954             Oakdale       NY      USA
143935  07/1947             Roswell       NM      USA
143936  06/1947      Corpus Christi       TX      USA
143937  06/1952              Auburn       WA      USA
143939  06/1950  Budapest (Hungary)  Unknown  Hungary

[143564 rows x 4 columns]


In [305]:
# based on unique values from country we clean country column and remove invalid values
unique_countries = ufo_report['country'].unique()
print(unique_countries)

# List of invalid values to remove
invalid_countries = ['no', 'unknown', 'none', 'not applicable', 'unknown/at sea', 'unavailable', 'in orbit', 'space', 'atlantic ocean', 'caribbean sea', 'pacific ocean', 'international space station', 'moon', 'mars', 'none', 'not found']

# Convert the 'Country' column to lowercase and filter out invalid values
ufo_report = ufo_report[~ufo_report['country'].str.lower().isin(invalid_countries)]

['USA' 'Unknown' 'United Kingdom' 'Canada' 'turkey' 'India' 'Australia'
 'Malta' 'Switzerland' 'France' 'Guam' 'Puerto Rico' 'Ukraine' 'Mexico'
 'Ireland' 'Japan' 'Germany' 'Poland' 'New Zealand' 'Pakistan'
 'South Africa' 'Papua New Guinea' 'Kenya' 'Thailand' 'Israel' 'Denmark'
 'Malaysia' 'Lebanon (Middle East)' 'Argentina' 'Myanmar' 'Cambodia'
 'Croatia' 'On the way to Cozumel' 'South-Africa' 'Brazil' 'Macedonia'
 'Cyprus' 'China' 'Panama' 'Romania' 'In Orbit' 'Trinidad/Tobago'
 'Jamaica' 'Luxemburg' 'Italy' 'East China Sea' 'Bulgaria' 'Iran' 'Spain'
 'Decalb' 'Portugal' 'Turkey' 'Indonesia' 'Lebanon' 'South Korea'
 'North Wales' 'Belgium' 'Netherlands' 'Luxembourg' 'Philippines'
 'Guatemala' 'Srui Lanka' 'Jordan' 'Afghanistan' 'Finland' 'Taiwan'
 'Algeria' 'Venezuela' 'Bahamas'
 'South Georgia and the south sandwich islands' 'Costa Rica' 'Honduras'
 'Lithuania' 'Bahrain' 'Palau' 'Western Australia' 'Mozambique'
 'Dominican Republic' 'Belize' 'Slovenia' 'Bosnia and Herzegovina'
 'Co

In [306]:
# create Country_Code column
# Funktion zum Abrufen des ISO 3166-1 Alpha-2 Ländercodes
def get_country_code(country_name):
    if isinstance(country_name, str):  # Prüfen, ob es eine Zeichenkette ist
        country = pycountry.countries.get(name=country_name)
        return country.alpha_2 if country else None
    return None  # Falls der Wert kein String ist (z. B. NaN)

# Beispiel-Datenframe erstellen (falls noch nicht vorhanden)
# ufo_report = pd.read_csv("deine_datei.csv")  # Falls Daten aus einer CSV geladen werden

# Neue Spalte "Country_Code" erstellen
ufo_report['country_code'] = ufo_report['country'].apply(lambda x: get_country_code(x) 
    if isinstance(x, str) and x.lower() not in ['non applicable', 'unknown', 'in orbit', 'at sea'] else None)

# Zeilen mit fehlendem "Country_Code" entfernen
ufo_report = ufo_report.dropna(subset=['country_code'])

In [ ]:
# Merge ufo_report with countries_df to get the corresponding country_id
ufo_report = ufo_report.merge(countries_df[['handle', 'country_id']], how='left', left_on='country_code', right_on='handle')

# Replace the country_code column with the country_id column
ufo_report['country_code'] = ufo_report['country_id']

# Drop unnecessary columns
ufo_report.drop(columns=['handle', 'country_id', "country"], inplace=True)
ufo_report = ufo_report.rename(columns={"country_code":"country_id"})

,date,city,state,country_id
0,06/2021,Felpham/Bognor Regis (UK/England),Unknown,79
1,10/2014,Swindon (UK/England),Unknown,79
2,04/2020,Brocket (Canada),AB,39
3,08/2013,Lincoln (Lincolnshire) (UK/England),Unknown,79
4,08/2014,Port Coquitlam (Canada),BC,39
...,...,...,...,...
14581,01/1958,Birchy Bay (Canada),NF,39
14582,04/1957,Lahore,Unknown,172
14583,10/1955,Chester (UK/England),Unknown,79
14584,10/1952,Fukuoka (Japan),Unknown,115


# Subscribers Cleaning

In [308]:
subscribers = pd.read_csv(r"C:\Users\Martijn\Downloads\subscribers_netflix_2024 (1).csv")

In [309]:
subscribers = subscribers.rename(columns={'estimated_subscribers': 'subscribers'}).reset_index(drop=True)


In [310]:
# Merge ufo_report with countries_df to get the corresponding country_id
subscribers = subscribers.merge(countries_df[['handle', 'country_id']], how='left', left_on='country_code', right_on='handle')

# Replace the country_code column with the country_id column
subscribers['country_code'] = subscribers['country_id']

# Drop unnecessary columns
subscribers.drop(columns=['handle', 'country_id', "country"], inplace=True)
subscribers = subscribers.rename(columns={"country_code":"country_id"})

In [356]:
# Get unique entries from both columns
unique_dates_ufo = ufo_report["date"].unique()
unique_dates_movies = movies["date_added_formatted"].unique()

# Combine and get all unique entries
all_unique_dates = pd.unique(np.concatenate((unique_dates_ufo, unique_dates_movies)))
# Create a mapping of dates to their indices in all_unique_dates
date_to_index = {date: idx for idx, date in enumerate(all_unique_dates)}

# Replace dates in ufo_report["date"] with their corresponding indices
ufo_report["date_index"] = ufo_report["date"].map(date_to_index)

# Replace dates in movies["date_added_formatted"] with their corresponding indices
movies["date_index"] = movies["date_added_formatted"].map(date_to_index)

In [361]:
countries_df

,country_id,handle,name
0,0,AW,Aruba
1,1,AF,Afghanistan
2,2,AO,Angola
3,3,AI,Anguilla
4,4,AX,Åland Islands
...,...,...,...
244,244,WS,Samoa
245,245,YE,Yemen
246,246,ZA,South Africa
247,247,ZM,Zambia


In [359]:
date_to_index

{'06/2021': 0,
 '10/2014': 1,
 '04/2020': 2,
 '08/2013': 3,
 '08/2014': 4,
 '07/2011': 5,
 '07/2010': 6,
 '06/2012': 7,
 '09/2012': 8,
 '06/2014': 9,
 '10/2013': 10,
 '07/2015': 11,
 '09/2013': 12,
 '12/2013': 13,
 '11/2015': 14,
 '08/2012': 15,
 '09/2015': 16,
 '10/2019': 17,
 '07/2012': 18,
 '09/2014': 19,
 '07/2013': 20,
 '07/2014': 21,
 '03/2020': 22,
 '11/2013': 23,
 '11/2012': 24,
 '08/2020': 25,
 '12/2019': 26,
 '11/2019': 27,
 '07/2016': 28,
 '10/2012': 29,
 '04/2014': 30,
 '12/2012': 31,
 '01/2014': 32,
 '10/2015': 33,
 '07/2020': 34,
 '06/2013': 35,
 '10/2011': 36,
 '08/2011': 37,
 '09/2019': 38,
 '07/2009': 39,
 '05/2014': 40,
 '01/2020': 41,
 '02/2020': 42,
 '08/2015': 43,
 '09/2016': 44,
 '07/2008': 45,
 '09/2009': 46,
 '01/2012': 47,
 '05/2020': 48,
 '07/2019': 49,
 '08/2016': 50,
 '09/2011': 51,
 '08/2004': 52,
 '08/2003': 53,
 '08/2010': 54,
 '01/2015': 55,
 '10/2008': 56,
 '09/2005': 57,
 '05/2013': 58,
 '05/2012': 59,
 '09/2022': 60,
 '03/2012': 61,
 '10/2020': 62,
 '